In [103]:
from labeler import *

data = getLabels("JawClench_labels_Ansh_12-02-21-1918.txt")
startTime = data[0]
labels = data[1]
data = getData("OpenBCI-RAW-2021-12-02_19-19-53.txt",labels,startTime)

data2 = getLabels("JawClench_labels_Ansh_12-02-21-1930.txt")
startTime = data2[0]
labels = data2[1]
data2 = getData("OpenBCI-RAW-2021-12-02_19-31-12.txt", labels, startTime)

data3 = getLabels("Recordings\Spring_2022\Evan_JawClench_1.txt")
startTime = data3[0]
labels = data3[1]
data3 = getData("Recordings\Spring_2022\OpenBCISession_2022-02-16_Evan_JawClench_2\OpenBCI-RAW-2022-02-16_19-30-28.txt", labels, startTime)

data = np.append(data, data2, axis=0).tolist()
data = np.append(data, data3, axis=0).tolist()


for i in range (len(data)):
    for j in range(len(data[i])):
        if j < len(data[i])-1:
            data[i][j] = (float)(data[i][j])
        else:
            data[i][j] = (int)(data[i][j])
data = np.array(data)

2021-12-02 19:19:55.073396
2021-12-02 19:31:14.177762
2022-02-16 19:30:30.103622


In [104]:
from sklearn import svm
bsize = 200
data = np.array(data)
X = np.zeros((len(data)//bsize, 4))
y = np.zeros(len(data)//bsize)
for i in range(len(data)//bsize):
    curX = data[bsize*i: bsize*(i+1), :-1]
    curY = data[bsize*i: bsize*(i+1), -1:]
    y[i] = np.rint(np.mean(curY))
    averages = (np.mean(curX, axis=0, keepdims=True))
    mean_deviation = np.mean(curX - np.matmul(np.ones((bsize, 1)), (averages)), axis=0)
    X[i] = mean_deviation

In [143]:
print(X.shape)
print(y.shape)

X

(123, 4)
(123,)


array([[ 3.18323146e-14,  2.72848411e-14, -3.63797881e-14,
        -3.63797881e-14],
       [-5.45696821e-14, -1.13686838e-14,  4.54747351e-15,
        -9.09494702e-14],
       [-1.43245416e-13,  1.46371804e-13, -1.37561074e-13,
        -1.45519152e-13],
       [-9.09494702e-15,  2.72848411e-14, -1.42108547e-14,
        -6.36646291e-14],
       [ 0.00000000e+00,  3.91082722e-13, -4.72937245e-13,
         4.09272616e-13],
       [ 7.27595761e-14,  2.34479103e-14,  1.81898940e-14,
         2.18278728e-13],
       [-9.09494702e-14,  2.04636308e-14,  2.27373675e-14,
        -4.54747351e-14],
       [-9.09494702e-14,  3.06954462e-14, -3.24007488e-14,
        -1.81898940e-13],
       [ 3.22870619e-13, -1.03227649e-12,  1.95541361e-13,
         9.20863386e-14],
       [ 0.00000000e+00,  1.36992639e-13, -9.20863386e-14,
        -3.63797881e-14],
       [-5.45696821e-14, -5.68434189e-14,  0.00000000e+00,
         9.09494702e-14],
       [-1.42108547e-13,  1.96791916e-12, -2.62616595e-13,
      

In [133]:
from sklearn.model_selection import train_test_split

X_res, X_test, y_res, y_test = train_test_split(X, y, test_size=0.4, random_state = 4)
X = X_res
y = y_res
model = svm.SVC(probability=True)
model.fit(X, y)

SVC(probability=True)

In [142]:
correct = 0
total = 0

# X = np.concatenate((X_test, X))
# y = np.concatenate((y_test, y))

print((model.predict_proba(X_test)[:,1]))
print(y_test)
val_error = np.mean(np.abs((model.predict_proba(X_test)[:,1]) - y_test))
print(val_error)

X = np.concatenate((X_test, X_res))
y = np.concatenate((y_test, y_res))

overall_error = np.mean(np.abs(model.predict_proba(X)[:,1] - y))
print(overall_error)
        

[0.5        0.55551614 0.43916249 0.50530918 0.47881105 0.58035844
 0.56212067 0.50601284 0.5163619  0.41687488 0.65240197 0.47627731
 0.56600518 0.39526763 0.64570327 0.66009086 0.57812166 0.52037061
 0.55138624 0.51526503 0.39397087 0.74405601 0.51458555 0.37414851
 0.53363552 0.60959361 0.6893846  0.57421976 0.48445311 0.60712557
 0.42515088 0.61840366 0.56757158 0.60061334 0.5        0.55176542
 0.48138028 0.56225163 0.46117717 0.38674997 0.57278596 0.52423422
 0.48049764 0.65997015 0.59997548 0.69768589 0.51108353 0.34309628
 0.6459352  0.52513524]
[0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0.
 1. 1.]
0.4979057070551729
0.5299564614179021
